In [ ]:
#TBD
#https://python.langchain.com/docs/get_started/quickstart/

In [2]:
# #%pip install python-dotenv
# # Upgrade langchain
# %pip install --upgrade langchain
# # Install anthropic support
# %pip install langchain-anthropic
# # Uninstall pydantic
# %pip uninstall pydantic -y
# # Re install pydantic specific version of pydantic (original version ==1.10.14)
# %pip install pydantic==1.10.14
# # %pip show langchain-anthropic
# # %pip show anthropic
# # %pip show defusedxml
# # %pip show langchain-core

In [1]:
#%pip install python-dotenv
from dotenv import load_dotenv
import os
import langchain

In [2]:
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variables
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')

In [ ]:
# https://python.langchain.com/docs/guides/development/pydantic_compatibility/
# Uninstall pydantic
# %pip uninstall pydantic -y
# # Install specific version of pydantic (original version ==1.10.14)
# %pip install pydantic==1.10.14
# # Install langchain-anthropic
#%pip install langchain-anthropic


In [6]:
# I don't think I need this anymore
#%pip uninstall langchain-anthropic -y
#%pip install langchain-anthropic # current version 0.1.8
#from langchain_anthropic import ChatAnthropic
#%pip show pydantic # Installed version ==1.10.14, current version ==2.7.0
#%pip show langchain-anthropic
#%pip install --upgrade pydantic
#%pip show langchain

In [3]:
# Now import
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
llm = ChatAnthropic(model="claude-3-sonnet-20240229",
                    api_key=anthropic_api_key,
                    temperature=0.2,
                    max_tokens=1024)

In [5]:
response = llm.invoke("how can langsmith help with testing?")
print(type(response))

<class 'langchain_core.messages.ai.AIMessage'>


In [6]:
# Check out the type:
print(type(response))
# Show the raw contents
response

<class 'langchain_core.messages.ai.AIMessage'>


AIMessage(content="Langsmith can help with testing in several ways:\n\n1. **Test Case Generation**: Langsmith can be used to automatically generate test cases based on the requirements or specifications of the software being tested. It can analyze the code and generate test cases that cover different scenarios, edge cases, and boundary conditions.\n\n2. **Test Data Generation**: Langsmith can be used to generate realistic and diverse test data for testing purposes. This can include generating data for databases, files, or other input sources required by the software under test.\n\n3. **Test Automation**: Langsmith can be used to write and maintain automated test scripts. Its natural language understanding capabilities can help translate test requirements or scenarios into executable test scripts, reducing the effort required for manual test script creation.\n\n4. **Test Reporting**: Langsmith can assist in generating human-readable test reports by analyzing and summarizing the test res

In [7]:
# Inside this object, I can get to the content, but I have to know where to look
print(response.content)

Langsmith can help with testing in several ways:

1. **Test Case Generation**: Langsmith can be used to automatically generate test cases based on the requirements or specifications of the software being tested. It can analyze the code and generate test cases that cover different scenarios, edge cases, and boundary conditions.

2. **Test Data Generation**: Langsmith can be used to generate realistic and diverse test data for testing purposes. This can include generating data for databases, files, or other input sources required by the software under test.

3. **Test Automation**: Langsmith can be used to write and maintain automated test scripts. Its natural language understanding capabilities can help translate test requirements or scenarios into executable test scripts, reducing the effort required for manual test script creation.

4. **Test Reporting**: Langsmith can assist in generating human-readable test reports by analyzing and summarizing the test results. It can highlight impo

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [9]:
# Chain
chain = prompt | llm 

In [10]:
# Invoke
myInput= {"input": "how can langsmith help with testing?"}
#response = chain.invoke({"input": "how can langsmith help with testing?"})
response = chain.invoke(myInput)

In [11]:
print(response.content)

Langsmith can be a valuable tool for testing software applications and systems. Here are some ways Langsmith can assist with testing:

1. **Test Case Generation**: Langsmith's natural language processing capabilities can be used to generate test cases from requirements documents, user stories, or other textual descriptions of the system's expected behavior. This can help automate the process of creating comprehensive test suites.

2. **Test Data Generation**: Langsmith can analyze the application's data requirements and generate realistic test data, including edge cases and boundary conditions. This can be particularly useful for testing data-intensive applications or scenarios where manually creating test data is time-consuming or error-prone.

3. **Test Automation**: Langsmith can be integrated with testing frameworks and tools to automate the execution of test cases. By understanding natural language instructions, Langsmith can translate high-level test scenarios into executable tes

In [12]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [13]:
# Defimne the 3-step-chain
chain = prompt | llm | output_parser

In [14]:
# Now the response is the result of the 3-steps
response = chain.invoke(myInput)

In [15]:
print(response)

LangSmith can be a valuable tool for testing in several ways:

1. **Test Case Generation**: LangSmith can be used to generate test cases automatically based on the requirements or specifications provided. It can analyze the input and generate a comprehensive set of test cases, covering various scenarios and edge cases.

2. **Test Data Generation**: LangSmith can generate realistic and diverse test data for testing purposes. This can be particularly useful when testing with large datasets or when dealing with complex data structures.

3. **Test Script Generation**: LangSmith can generate test scripts in various programming languages, such as Python, Java, or JavaScript. These scripts can be used to automate the testing process, reducing the need for manual testing and increasing efficiency.

4. **Natural Language Test Case Description**: LangSmith can understand natural language descriptions of test cases and convert them into executable test scripts or test case specifications. This ca